<a href="https://colab.research.google.com/github/Alecia113/NLP-Emotional-analysis/blob/main/AE1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AE1

In [1]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1lTD6bgRkmwguGAr30v-r0KBPdtnVneLb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.pkl')  

id = '1pCUdlZMoj99UZHtqFeza86fvVQfFmDFX'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.pkl')  

import pickle
training_data = pickle.load(open("training_data.pkl","rb"))
testing_data = pickle.load(open("testing_data.pkl","rb"))

print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data))) #8000
print("Size of testing dataset: {0}".format(len(testing_data))) #2000
print("------------------------------------")

print("------------------------------------")
print("Sample Data")
print("LABEL: {0} / SENTENCE: {1}".format(training_data[0][0], training_data[0][1]))
print("------------------------------------")
#LABEL: neg / SENTENCE: hopeless for tmr :(

------------------------------------
Size of training dataset: 8000
Size of testing dataset: 2000
------------------------------------
------------------------------------
Sample Data
LABEL: neg / SENTENCE: hopeless for tmr :(
------------------------------------


In [2]:
import torch
#You can enable GPU here (cuda); or just CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 1.2. Preprocess data



*You are required to describe which data preprocessing techniques were conducted with justification of your decision. *
请您说明采用了哪些数据预处理技术，并说明决定的理由。*

In [3]:
testing_data[0][1]

'@AndreaMarySmith very helpful .... Or will be once I stop crying :('

In [4]:
#先搞个label。再搞个sentence
#train_data = [training_data[i][1] for i in range(len(training_data))]

##print(train_data[0]) #hopeless for tmr :(
##test:'@AndreaMarySmith very helpful .... Or will be once I stop crying :('
'''
train_data =[]
for i in range(len(training_data)):
  train_data.append(training_data[i][1])
print(train_data[0]) 
#上下一样
'''

train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]

test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))] #neg pos


In [ ]:
#做预处理 变小写
text_train = [s.lower() for s in train_data]
text_test = [s.lower() for s in test_data]

import pprint
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()


train_t=[]
for s in text_train:
  text_train = tknzr.tokenize(s)                         #每句话分别作了分词 text_train[0]第一句话的分词
  train_t.append(text_train)  
#pprint.pprint(train_t[:10])         #:(    :-(


test_t=[]     #2000
for w in text_test:
  text_test = tknzr.tokenize(w)
  test_t.append(text_test)
#pprint.pprint(text_test[1:3])

'''
#做token分词处理 #还是有问题 要改成那种 要
import nltk
nltk.download('punkt')
import pprint
from nltk.tokenize import word_tokenize

train_t=[]
for s in text_train:
  text_train = word_tokenize(s)                         #每句话分别作了分词 text_train[0]第一句话的分词
  train_t.append(text_train)  
pprint.pprint(train_t[:10])             #目前是8000

test_t=[]     #2000
for w in text_test:
  text_test = word_tokenize(w)
  test_t.append(text_test)
#pprint.pprint(text_test[1:3])
'''
'''
import re
def remove_punctuation_re(x):
    x = re.sub(r'[^\w\s]','',str(x))  #删标点。
    return x
#删掉标点

import re
def remove_punctuation_re(x):
  for i in range(len(x)):
    for x in x[i]:
      x = re.sub(r'[^\w\s]','',x)
      return x
  
text_train = [remove_punctuation_re(s) for s in train_t]
text_test = [remove_punctuation_re(s) for s in test_t]
pprint.pprint(train_t)
'''

In [ ]:
#那我取出最后-1的词呗不行。还是要保留表情。

In [ ]:
#停用词  
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stop_words = sw.words()

text_train_ns=[]    #8000
for tokens in train_t:
    filtered_sentence1 = [w for w in tokens if not w in stop_words]
    text_train_ns.append(filtered_sentence1)
pprint.pprint(text_train_ns[:10])

text_test_ns=[]   #2000
for tokens in test_t:
  filtered_sentence2 = [w for w in tokens if not w in stop_words]
  text_test_ns.append(filtered_sentence2)


In [25]:
#找词根
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

text_train_le = []
for tokens in text_train_ns:
  lemma_sentence1 = [lemmatizer.lemmatize(w) for w in tokens ]
  text_train_le.append(lemma_sentence1)
pprint.pprint(text_train_le[:10])

text_test_le = []
for tokens in text_test_ns:
  lemma_sentence2 = [lemmatizer.lemmatize(w) for w in tokens]
  text_test_le.append(lemma_sentence2)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[['hopeless', 'tmr', ':('],
 ['everything',
  'kid',
  'section',
  'ikea',
  'cute',
  '.',
  'shame',
  "i'm",
  'nearly',
  '19',
  '2',
  'month',
  ':('],
 ['@hegelbon', 'heart', 'sliding', 'waste', 'basket', '.', ':('],
 ['“',
  '@ketchburning',
  ':',
  'hate',
  'japanese',
  'call',
  '"',
  'ban',
  '"',
  ':(',
  ':(',
  '”'],
 ['dang', 'starting', 'next', 'week', '"', 'work', '"', ':('],
 ['oh', 'god', ',', 'baby', "'", 'face', ':(', 'https://t.co/9fcwgvaki0'],
 ['@rileymcdonough', 'make', 'smile', ':(', '('],
 ['@f0ggstar',
  '@stuartthull',
  'work',
  'neighbour',
  'motor',
  '.',
  'asked',
  'said',
  'hate',
  'update',
  'search',
  ':(',
  'http://t.co/xvmtuikwln'],
 ['?',
  ':(',
  '"',
  '@tahuodyy',
  ':',
  'sialan',
  ':(',
  'https://t.co/hv1i0xcrl2',
  '"'],
 ['athabasca',
  'glacier',
  '#1948',
  ':-(',
  '#athabasca',
  '#glacier',
  '#jasper

In [26]:
#去掉复数
from nltk.stem.porter import *
stemmer = PorterStemmer()
#text_train_le = [text_train_le[i] for i in range(len(text_train_le))]
#singles = [stemmer.stem(plural) for plural in text_train_le[i] for i in range(len(text_train_le))]
for i in range(len(text_train_le)):
  for plural in text_train_le[i]:
    singles = stemmer.stem(plural)
#print(text_train_le)

#text_train_le = [text_train_le[i] for i in range(len(text_train_le))]
#singles = [stemmer.stem(plural) for plural in text_train_le[i] for i in range(len(text_train_le))]
for i in range(len(text_test_le)):
  for plural in text_test_le[i]:
    singles = stemmer.stem(plural)
print(text_test_le)


[['@andreamarysmith', 'helpful', '...', 'stop', 'cry', ':('], ['@realyys_', 'otl', 'nevermind', ':(', 'least', 'got', 'jeon'], ['soon', 'tweeted', 'planted', 'claw', 'thigh', 'traction', 'zoomed', 'away', ':('], ['@luketothestars', 'damnit', ':('], ['@klm', 'used', 'pry', '/', 'pv', '...', 'wish', 'could', 'relive', 'day', 'become', 'nyc', '/', 'pv', 'buy', 'way', 'communicate', 'nyc', 'usa', 'klm', ':('], ['really', 'hot', ':-('], ['@carouselballet', 'monday', '?', ':('], ['@badoeuf', 'going', 'stop', 'breakfast', 'early', 'might', 'remove', '11am', 'website', '.', 'even', "mcd's", 'pull', 'trick', '.', ':('], ['@blairforce2', 'mean', 'way', ':(', '3rd', 'load', 'hung'], [':(', 'wtf', 'supposed', 'without'], ['headache', 'strike', ':('], ['english', 'weather', 'need', 'fix', ':(', '('], ['live', 'fam', 'bam', 'cough', ':('], ['absolutely', 'gutted', 'james', 'bay', 'ticket', 'sold', 'manchester', ':('], ['seventh', 'spot', 'lang', ':(', '#otwolgrandtrailer'], ['splendour', ':('], ['@l

现在train： text_train_le; test : text_test_le;
label = train_label test = test_label

# 2 - Model Implementation

In [ ]:
# word embedding  2分 [lab2]
'''
  创建word embedding model(表现word vectors eg: word2vec-CBOW, word2vec-Skip gram, fastText + Glove[lab2])
  input embedding of your sequence model
  [lab3,4]使用了one-hot vectors 作为inputs 给长序列模型
  【要求做的】
    1.数据的预处理（word embeddings[lab2;还要做解释说明]） NLTK Twitter dataset ( section 1 提供了) or/and 任何数据集（TED talk , Google News)
    2.创建训练模型（for word embeddings) 创建训练模型；需要说清楚超参数【lab2（dimension of embeddings; window size, learning rate,etc.】 
        note 任何word embeddings model[lab2] (word2vec-CBOW, word2vec-Skip gram, fasttext, glove)【证明】
    3.训练模型： train model
    '''

In [ ]:
id2 = '0B7XkCwpI5KDYNlNUTTlSS21pQmM'
downloaded = drive.CreateFile({'id':id2}) 
downloaded.GetContentFile('GoogleNews-vectors-negative300.bin.gz') 

In [ ]:
# Uncompress the downloaded file
!gzip -d /content/GoogleNews-vectors-negative300.bin.gz

In [ ]:
from gensim.models import KeyedVectors
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the pretrained vectors with KeyedVectors instance - might be long waiting! 
filename = 'GoogleNews-vectors-negative300.bin'
gn_wv_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
# Now we can try to calculate "King - Man + Woman = ?" again
result = gn_wv_model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)



In [ ]:
from gensim.models import FastText #fastText-Skip Gram
# Now we initialize and train FastText with Skip Gram architecture (sg=1)
ft_sg_model = FastText(sentences, size=100, window=5, min_count=5, workers=2, sg=1)
# As we can see, FastText allows us to obtain word vectors for out-of-vocabulary words
result=ft_sg_model.wv.most_similar("electrofishing")
pprint.pprint(result)

## 2.1. Word Embeddings

In [ ]:
#模块实现 model implementation 7分
'''
【要做的】
  1，word embedding module, 2，Lexicon Embedding module + 3，Bi-directional RNN sequence model

  [从lab2,4,5随便选参数]embeddings_dimension ; learning_rate epochs;
  [lab5 有模型构建]


# word embedding  2分 [lab2]
  创建word embedding model(表现word vectors eg: word2vec-CBOW, word2vec-Skip gram, fastText + Glove[lab2])
  input embedding of your sequence model
  [lab3,4]使用了one-hot vectors 作为inputs 给长序列模型
  【要求做的】
    1.数据的预处理（word embeddings[lab2;还要做解释说明]） NLTK Twitter dataset ( section 1 提供了) or/and 任何数据集（TED talk , Google News)
    2.创建训练模型（for word embeddings) 创建训练模型；需要说清楚超参数【lab2（dimension of embeddings; window size, learning rate,etc.】 
        note 任何word embeddings model[lab2] (word2vec-CBOW, word2vec-Skip gram, fasttext, glove)【证明】
    3.训练模型： train model

#Lexicon embedding 2分 （词典嵌入）
  每个单词的积极或者消极的lexicon【从文件夹下载：opinion Lexicaon】(2006 positive and 4783 negative words)
  每个单词要被改成： one-dimensional一维分类嵌入，三类。【eg:[not_exist(0), negative(1), positive(2);这个012类别 会在[2.3 的Bi-directional RNN 模型中用到]]
  【如果你想用超过一维，或者不使用分类嵌入（categorical） 证明观点】

# Bi-directional RNN Sequence Model 3分
Many-to-One (N to 1) 建个n to 1 sequence model 为了探测感情（detect sentiment/emotion）你的模型应该是评价中选出的最佳模型（将在第3节评价【evaluation】中讨论）。
  【做的】
    1.apply/import word + lexicon embedding as input: 你要把训练好的词嵌入和词库嵌入合并起来，并应用到序列模型上    ##不太理解
    2.1构建训练Sequence model: 构建Bi-directional RNN-based(Bi-RNN / Bi-LSTM / Bi-GRU) 
    2.2Many to One (N to 1) sequence model (N: word, One: sentiment- positive or negative)
    3.描述超参【证明】【lab4,5】[the number of epochs, learning rate ……]
    4.Train model  展示：training loss + number of epochs[lab4,lab5]

    【不展示没分】
'''
'''
【要做的】
  1，word embedding module, 2，Lexicon Embedding module + 3，Bi-directional RNN sequence model

  [从lab2,4,5随便选参数]embeddings_dimension ; learning_rate epochs;
  [lab5 有模型构建]


# word embedding  2分 [lab2]
  创建word embedding model(表现word vectors eg: word2vec-CBOW, word2vec-Skip gram, fastText + Glove[lab2])
  input embedding of your sequence model
  [lab3,4]使用了one-hot vectors 作为inputs 给长序列模型
  【要求做的】
    1.数据的预处理（word embeddings[lab2;还要做解释说明]） NLTK Twitter dataset ( section 1 提供了) or/and 任何数据集（TED talk , Google News)
    2.创建训练模型（for word embeddings) 创建训练模型；需要说清楚超参数【lab2（dimension of embeddings; window size, learning rate,etc.】 
        note 任何word embeddings model[lab2] (word2vec-CBOW, word2vec-Skip gram, fasttext, glove)【证明】
    3.训练模型： train model

 
    '''

*You are required to describe which model was implemented (i.e. Word2Vec with CBOW, FastText with SkipGram, etc.) with justification of your decision *
*要求您说明采用的是哪种模式（即Word2Vec与CBOW、FastText与SkipGram等），并说明您决定的理由*。【也就是三种都要做一做】

In [ ]:
# Please comment your code
#我希望你考虑一下，我们是否真的需要用标签来对推文进行分类。情感分析）;)
#如果你尝试将TED或Google News与ltktwitter数据集一起加入到训练单词嵌入的数据中，准确率会提高更多，但过低的准确率会使你很难比较不同超参数的结果
#显示最好的参数
#对于你的Q1，是的，而且你需要利用lab2中学习的知识来建立单词嵌入的训练模型。
#对于你的Q2和Q3，你的理解也是正确的，你需要将单词嵌入和词库嵌入连在一起作为基于双向RNN的序列模型的输入。
#【为什么要给测试集做预处理？】
#如何定义标点，如何定义表情。 tokenize 标点是一个 表情是两个（或者大于）

### 2.1.1. Data Preprocessing for Word Embeddings 字词嵌入的数据预处理

*You are required to describe which preprocessing techniques were used with justification of your decision.*
您需要说明使用了哪些预处理技术，并说明您的决定的理由。

In [ ]:
# Please comment your code


### 2.1.2. Build Word Embeddings Model. 建立单词嵌入模型

*You are required to describe how hyperparameters were decided with justification of your decision.*您需要说明超参数是如何决定的，并说明决定的理由。

In [ ]:
# Please comment your code

### 2.1.3. Train Word Embeddings Model

In [ ]:
# Please comment your code

#Test


In [ ]:
testing_data[0]#('neg', '@AndreaMarySmith very helpful .... Or will be once I stop crying :(')
'''
都是标签+句子
'''

In [ ]:
training_data[0]#('neg', 'hopeless for tmr :(')
training_data[1]#('neg',"Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(")
training_data[2]#('neg', '@Hegelbon That heart sliding into the waste basket. :(')

In [ ]:
len(testing_data)

In [ ]:
testing_data  #2000
'''
 "@NUFC_Vine he's gona miss Sheff Utd/York probably waiting for a visa which leaves gim with 1 friendly. Bet he starts season on the bench :("),
 ('neg', "@ranaPTX_ BUT WHO WILL LEAD US WHEN YOU'RE GONE :(("),
 ('neg', '@LottyStorer @charliee_catlin 😂😩😩 what a mess :('),
 ('neg', '@twinitisha irony is its more harmful .. :('),
 ('neg', '@DollyyDaydreams I miss you so much more :( xxx'),
 ('neg', 'no motivation :('),
 ('neg',
  '@LiLGeekette but that is the point. Because it is YOLO I sort of die when lactose is in me... Ugh like 3 months pregnant bloating :('),
 (
'''

In [ ]:
len(training_data)

In [ ]:
training_data #8000
'''
 ('neg', "@rupexo @LordOfTheMics fuckkk it's a Wednesday :("),
 ('neg',
  '@PetiteMistress DO IT! I want to start one for making small games, but I feel like I need to get a jump start before asking for support :('),
 ('neg', 'Justin where are you ? :( @justinbieber'),
 ('neg', ":( :( I'm gonna cry"),
 ('neg',
  "@IanHallard @ArtsTheatreLDN I'm really excited but also really sad because once again I won't be able to see it :("),
 ('neg', "I can't finish my Sanum today! :(\n#LLAOLLAO #Dessert but too full"),
 ('neg', '@jxstkatie @TheSvante i want foood :(('),
 ('neg', 'Last day :('),
 ('neg', '#UberIceCream was super! But we didnt get the glares :( @Uber_Pune'),
 ('neg', "@ffyeahh i don't know how to do vines :("),
 ('neg',
  "@selenagomez you was Tweetin earlier this morning &amp; i wasn't here :( sad mood all the day :("),
 ('n
 '''